In [1]:
import numpy as np
import pandas as pd
import os
import featuretools as ft
from shl.prepare import normalize_epoch_time

%load_ext autoreload
%autoreload 2

In [2]:
train_wifi = normalize_epoch_time(pd.read_parquet('../data/train/WiFi.parquet'), 'Epoch time [ms]')
train_gps = normalize_epoch_time(pd.read_parquet('../data/train/GPS.parquet'), 'Epoch time [ms]')
train_location = normalize_epoch_time(pd.read_parquet('../data/train/Location.parquet'), 'Epoch time [ms]')
train_label = normalize_epoch_time(pd.read_parquet('../data/train/Label.parquet'), 'epoch_time')

In [9]:
train_wifi.dtypes

Epoch time [ms]     int64
Ignore1             int64
Ignore2             int64
Unknown             int64
BSSID              object
SSID               object
RSSI                int64
Frequency [MHz]     int64
Capabilities       object
epoch_time_id       int64
dtype: object

In [2]:
train_label

NameError: name 'train_label' is not defined

In [3]:
es = ft.EntitySet(id="train")
es = es.entity_from_dataframe(
    entity_id="label",
    dataframe=train_label,
    index='epoch_time_id',
    time_index="epoch_time",
    variable_types={"label": ft.variable_types.Categorical},
)
es

Entityset: train
  Entities:
    label [Rows: 980527, Columns: 3]
  Relationships:
    No relationships

In [4]:
es.entity_from_dataframe(
    entity_id='location',
    dataframe=train_location,
    index='id',
    make_index=True,
    time_index='Epoch time [ms]',
    variable_types={
        "epoch_time_id": ft.variable_types.Id,
        "Accuracy": ft.variable_types.Numeric,
        "Latitude": ft.variable_types.Numeric,
        "Longitude": ft.variable_types.Numeric,
        "Altitude": ft.variable_types.Numeric,
    },
)
es

Entityset: train
  Entities:
    label [Rows: 980527, Columns: 3]
    location [Rows: 911109, Columns: 9]
  Relationships:
    No relationships

In [5]:
es.entity_from_dataframe(
    entity_id='wifi',
    dataframe=train_wifi,
    index='id',
    make_index=True,
    time_index='Epoch time [ms]',
    variable_types={
        "epoch_time_id": ft.variable_types.Id,
        "BSSID": ft.variable_types.Id,
        "SSID": ft.variable_types.Id,
        "RSSI": ft.variable_types.Numeric,
        "Frequency [MHz]": ft.variable_types.Numeric,
        "Capabilities": ft.variable_types.NaturalLanguage,
    },
)
es

Entityset: train
  Entities:
    label [Rows: 980527, Columns: 3]
    location [Rows: 911109, Columns: 9]
    wifi [Rows: 12604135, Columns: 11]
  Relationships:
    No relationships

In [6]:
es.entity_from_dataframe(
    entity_id='gps',
    dataframe=train_gps,
    index='id',
    make_index=True,
    time_index='Epoch time [ms]',
    variable_types={
        "epoch_time_id": ft.variable_types.Id,
        "ID": ft.variable_types.Id,
        "SNR": ft.variable_types.Numeric,
        "Azimuth [degrees]": ft.variable_types.Numeric,
        "Elevation [degrees]": ft.variable_types.Numeric,
    },
)
es

Entityset: train
  Entities:
    label [Rows: 980527, Columns: 3]
    location [Rows: 911109, Columns: 9]
    wifi [Rows: 12604135, Columns: 11]
    gps [Rows: 14046969, Columns: 9]
  Relationships:
    No relationships

In [7]:
es = es.add_relationship(
    ft.Relationship(es["label"]["epoch_time_id"], es["wifi"]["epoch_time_id"])
).add_relationship(
    ft.Relationship(es["label"]["epoch_time_id"], es["gps"]["epoch_time_id"])
).add_relationship(
    ft.Relationship(es["label"]["epoch_time_id"], es["location"]["epoch_time_id"])
)
es

Entityset: train
  Entities:
    label [Rows: 980527, Columns: 3]
    location [Rows: 911109, Columns: 9]
    wifi [Rows: 12604135, Columns: 11]
    gps [Rows: 14046969, Columns: 9]
  Relationships:
    wifi.epoch_time_id -> label.epoch_time_id
    gps.epoch_time_id -> label.epoch_time_id
    location.epoch_time_id -> label.epoch_time_id